In [1]:
from __future__ import print_function 
import serial
import argparse
import logging

In [2]:
def remove_crud(string):
    """Return string without useless information.

     Return string with trailing zeros after a decimal place, trailing
     decimal points, and leading and trailing spaces removed.
     """
    if "." in string:
        string = string.rstrip('0')

    string = string.lstrip('0 ')
    string = string.rstrip(' .')

    return string

In [3]:
class Chain(serial.Serial):
    """Create Chain object.

    Harvard syringe pumps are daisy chained together in a 'pump chain'
    off a single serial port. A pump address is set on each pump. You
    must first create a chain to which you then add Pump objects.

    Chain is a subclass of serial.Serial. Chain creates a serial.Serial
    instance with the required parameters, flushes input and output
    buffers (found during testing that this fixes a lot of problems) and
    logs creation of the Chain.
    """
    def __init__(self, port):
        serial.Serial.__init__(self,port=port, stopbits=serial.STOPBITS_TWO, parity=serial.PARITY_NONE, timeout=2)
        self.flushOutput()
        self.flushInput()
        logging.info('Chain created on %s' % port)

In [4]:
class PumpError(Exception):
    pass

In [174]:
class Pump33:
    """Create Pump object for Harvard Pump 33.

    Argument:
        Chain: pump chain

    Optional arguments:
        address: pump address. Default is 0.
        name: used in logging. Default is Pump 33.
    """
    def __init__(self, chain, address=0, name='Pump 33'):
        self.name = name
        self.serialcon = chain
        self.address = '{0:02.0f}'.format(address)
        self.mode = None
        self.diameter = None
        self.flowrate = None
        self.targetvolume = None

        """Query model and version number of firmware to check pump is
        OK. Responds with a load of stuff, but the last three characters
        are XXY, where XX is the address and Y is pump status. :, > or <
        when stopped, running forwards, or running backwards. Confirm
        that the address is correct. This acts as a check to see that
        the pump is connected and working."""
        try:
            self.write('VER')
            resp = self.read(11)

            if int(resp[-3:-1]) != int(self.address):
                raise PumpError('No response from pump at address %s' %
                                self.address)
            else:
                print (repr(resp))
                
        except PumpError:
            self.serialcon.close()
            raise

        logging.info('%s: created at address %s on %s' % (self.name,
                      self.address, self.serialcon.port))

    def __repr__(self):
        string = ''
        for attr in self.__dict__:
            string += '%s: %s\n' % (attr, self.__dict__[attr]) 
        return string

    def write(self,command):
        self.serialcon.write(self.address + command + '\r')

    def read(self,bytes=5):
        response = self.serialcon.read(bytes)

        if len(response) == 0:
            raise PumpError('%s: no response to command' % self.name)
        else:
            return response
        
    def setmode(self, mode):
        """Set pump mode.

        Pump 33 has 3 mode: Auto Stop, Proportional, and Continuous. 
        The Command for them are AUT, PRO, and CON, respectively.
        """
        
        # Check if the input is a valid mode
        if mode not in ['AUT', 'PRO', 'CON']:
            raise PumpError('%s: %s is not a mode name' %(self.name, mode))
            
        self.write('MOD' + mode)
        resp = self.read(5)
        
        if (resp[-1] == ':' or resp[-1] == '<' or resp[-1] == '>'):
            # check if mode has been set correctlry
            self.write('MOD')
            resp = self.read(15)

            returned_mode = resp[1:-4]
            
            # Check mode was set accurately
            if returned_mode != mode:
                logging.error('%s: set mode (%s) does not match mode'
                              ' returned by pump (%s)' % (self.name, mode, 
                              returned_mode))
            elif returned_mode == mode:
                self.mode = mode
                logging.info('%s: mode set to %s' % (self.name, self.mode))
        else:
            raise PumpError('%s: unknown response to setmode' % self.name)
    
    def setdiameter1(self, diameter):
        """Set syringe 1 diameter (millimetres).

        Pump 33 syringe diameter range is 0.1-50 mm. Note that the diameters
        are in the following format: ffffff (e.g. 44.755 or 0.3257) 
        """
        if diameter > 50 or diameter < 0.1:
            raise PumpError('%s: diameter %s mm is out of range' % (self.name,
                            str(diameter)))

        # TODO: Got to be a better way of doing this with string formatting
        diameter = str(diameter)

        # Pump only considers 2 d.p. - anymore are ignored
        if len(diameter) > 6:
            diameter = diameter[0:6]
            diameter = remove_crud(diameter)
            logging.warning('%s: diameter truncated to %s mm' % (self.name,
                            diameter))
        else:
            diameter = remove_crud(diameter)

        # Send command   
        self.write('DIAA' + diameter)
        resp = self.read(5)

        # Pump replies with address and status (:, < or >)        
        if (resp[-1] == ':' or resp[-1] == '<' or resp[-1] == '>'):
            # check if diameter has been set correctlry
            self.write('DIAA')
            resp = self.read(15)

            returned_diameter = resp[1:-4]
            
            # Check diameter was set accurately
            if float(returned_diameter) != float(diameter):
                logging.error('%s: set diameter (%s mm) does not match diameter'
                              'returned by pump (%s mm)' % (self.name, diameter, 
                                returned_diameter))
            elif float(returned_diameter) == float(diameter):
                self.diameter = float(returned_diameter)
                logging.info('%s: diameter set to %s mm' % (self.name,
                             self.diameter))
        else:
            raise PumpError('%s: unknown response to setdiameter1' % self.name)
            
    def setdiameter2(self, diameter):
        """Set syringe 2 diameter (millimetres) (only valid in Proportional 
        (PRO) mode).

        Pump 33 syringe diameter range is 0.1-50 mm. Note that the diameters
        are in the following format: ffffff (e.g. 44.755 or 0.3257) 
        """
        
        # Check if the pump is in Proportional mode
        self.write('MOD')
        resp = self.read(15)
        
        returned_mode = resp[1:-4]
        
        if returned_mode != 'PRO':
            raise PumpError('%s: Setting syringe 2 diameter is only valid in '
                            'Proportional mode' % self.name)
        
        if diameter > 50 or diameter < 0.1:
            raise PumpError('%s: diameter %s mm is out of range' % 
                            (self.name, str(diameter)))

        # TODO: Got to be a better way of doing this with string formatting
        diameter = str(diameter)

        # Pump only considers 2 d.p. - anymore are ignored
        if len(diameter) > 6:
            diameter = diameter[0:6]
            diameter = remove_crud(diameter)
            logging.warning('%s: diameter truncated to %s mm' % (self.name,
                            diameter))
        else:
            diameter = remove_crud(diameter)

        # Send command   
        self.write('DIAB' + diameter)
        resp = self.read(5)
        print (repr(resp))

        # Pump replies with address and status (:, < or >)        
        if (resp[-1] == ':' or resp[-1] == '<' or resp[-1] == '>'):
            # check if diameter has been set correctlry
            self.write('DIAB')
            resp = self.read(15)
            print (repr(resp))

            returned_diameter = resp[1:-4]
            
            # Check diameter was set accurately
            if float(returned_diameter) != float(diameter):
                logging.error('%s: set diameter (%s mm) does not match diameter'
                              ' returned by pump (%s mm)' % (self.name, diameter,
                              returned_diameter))
            elif float(returned_diameter) == float(diameter):
                self.diameter = float(returned_diameter)
                logging.info('%s: diameter set to %s mm' % (self.name,
                             self.diameter))
        else:
            raise PumpError('%s: unknown response to setdiameter' % self.name)
            
    def setflowrate1(self, flowrate):
        """Set syringe 1 flow rate (microlitres per minute).

        Flow rate is converted to a string. Pump 33 requires it to have 
        the format: ffffff.

        The pump will tell you if the specified flow rate is out of
        range. This depends on the syringe diameter. See Pump 33 manual.
        """
        flowrate = str(flowrate)

        if len(flowrate) > 6:
            flowrate = flowrate[0:6]
            flowrate = remove_crud(flowrate)
            logging.warning('%s: flow rate truncated to %s uL/min' % (self.name, 
                             flowrate))
        else:
            flowrate = remove_crud(flowrate)

        self.write('RATA' + flowrate + 'UM')
        resp = self.read(5)

        if (resp[-1] == ':' or resp[-1] == '<' or resp[-1] == '>'):
            # Flow rate was sent, check it was set correctly
            self.write('RATA')
            resp = self.read(15)

            returned_flowrate = remove_crud(resp[1:7])
            
            if float(returned_flowrate) != float(flowrate):
                logging.error('%s: set flowrate (%s uL/min) does not match'
                              'flowrate returned by pump (%s uL/min)' % 
                              (self.name, flowrate, returned_flowrate))
            elif float(returned_flowrate) == float(flowrate):
                self.flowrate = float(returned_flowrate)
                logging.info('%s: flow rate set to %s uL/min' % (self.name,
                              str(self.flowrate)))
        elif 'OOR' in resp:
            raise PumpError('%s: flow rate (%s uL/min) is out of range' %
                           (self.name, flowrate))
        else:
            raise PumpError('%s: unknown response' % self.name)
            

In [ ]:
    def infuse(self):
        """Start infusing pump."""
        self.write('RUN')
        resp = self.read(5)
        while resp[-1] != '>':
            if resp[-1] == '<': # wrong direction
                self.write('REV')
            else:
                raise PumpError('%s: unknown response to to infuse' % self.name)
            resp = self.serialcon.read(5)
        
        logging.info('%s: infusing',self.name)

    def withdraw(self):
        """Start withdrawing pump."""
        self.write('REV')
        resp = self.read(5)
        
        while resp[-1] != '<':
            if resp[-1] == ':': # pump not running
                self.write('RUN')
            elif resp[-1] == '>': # wrong direction
                self.write('REV')
            else:
                raise PumpError('%s: unknown response to withdraw' % self.name)
                break
            resp = self.read(5)

        logging.info('%s: withdrawing',self.name)

    def stop(self):
        """Stop pump."""
        self.write('STP')
        resp = self.read(5)
        
        if resp[-1] != ':':
            raise PumpError('%s: unexpected response to stop' % self.name)
        else:
            logging.info('%s: stopped',self.name)

    def settargetvolume(self, targetvolume):
        """Set the target volume to infuse or withdraw (microlitres)."""
        self.write('MLT' + str(targetvolume))
        resp = self.read(5)

        # response should be CRLFXX:, CRLFXX>, CRLFXX< where XX is address
        # Pump11 replies with leading zeros, e.g. 03, but PHD2000 misbehaves and 
        # returns without and gives an extra CR. Use int() to deal with
        if resp[-1] == ':' or resp[-1] == '>' or resp[-1] == '<':
            self.targetvolume = float(targetvolume)
            logging.info('%s: target volume set to %s uL', self.name,
                         self.targetvolume)
        else:
            raise PumpError('%s: target volume not set' % self.name)

    def waituntiltarget(self):
        """Wait until the pump has reached its target volume."""
        logging.info('%s: waiting until target reached',self.name)
        # counter - need it to check if it's the first loop
        i = 0
    
        while True:
            # Read once
            self.serialcon.write(self.address + 'VOL\r')
            resp1 = self.read(15)

            if ':' in resp1 and i == 0:
                raise PumpError('%s: not infusing/withdrawing - infuse or '
                                'withdraw first', self.name)
            elif ':' in resp1 and i != 0:
                # pump has already come to a halt
                logging.info('%s: target volume reached, stopped',self.name)
                break

            # Read again
            self.serialcon.write(self.address + 'VOL\r')
            resp2 = self.read(15)

            # Check if they're the same - if they are, break, otherwise continue
            if resp1 == resp2:
                logging.info('%s: target volume reached, stopped',self.name)
                break

            i = i+1

In [18]:
chain = Chain('COM4')

In [183]:
chain.isOpen()

True

In [182]:
chain.open()

In [181]:
chain.close()

In [184]:
p33 = Pump33(chain, address=0) 

'\n33V2.0\r\n0:'


In [185]:
p33.address

'00'

In [95]:
p33.setmode('PRO')

In [179]:
p33.setdiameter1(55)

ValueError: Unknown format code 's' for object of type 'int'

In [132]:
p33.setflowrate1(1000)

'\nOOR\r'


In [198]:
a = 'hua'

print ('%s is currently '
       'living in %f' % (a, 0.5))

hua is currently living in 0.500000


In [ ]:
p11 = pumpy.Pump(chain,address=1) 
p11.setdiameter(10)  # mm
p11.setflowrate(2000)  ## microL/min
p11.settargetvolume(200)  ## microL
p11.infuse()
p11.waituntiltarget()  ## blocks until target reached
p11.withdraw()
p11.waituntiltarget()

phd = pumpy.PHD2000(chain,address=4)
phd.setdiameter(24)
phd.setflowrate(600)
phd.infuse()
phd.stop()
phd.withdraw()
phd.stop()
phd.settargetvolume(100)
phd.infuse()
phd.waituntiltarget()

chain.close()